<a href="https://colab.research.google.com/github/ramayer/google-colab-examples/blob/main/Postgres_with_PostGIS_on_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Postgres with PostGIS on Google Colab

## Install postgres and create a GIS database on a Google Colab VM

In [ ]:
%%shell
# Install postgres, postgis, etc.
#
# Google Colab notebooks seem to only have postgres 10 available. 
# For other Jupyter Lab hosts, use this: 
# apt-get install -y postgresql-12-postgis-3-scripts  libproj19

apt-get install -qq -y postgresql postgresql-client postgis osm2pgsql libproj-dev proj-bin
apt-get install -qq -y postgresql-10-postgis-scripts libproj12
apt-get install -qq -y cgi-mapserver mapserver-bin libmapserver-dev ruby jq

echo localhost:5432:*:gis:'gis' > $HOME/.pgpass
chmod og-rwx $HOME/.pgpass
service postgresql start
echo "alter system set listen_addresses='0.0.0.0';" | sudo -u postgres psql
service postgresql restart
echo "do \$\$ BEGIN create user gis password 'gis' SUPERUSER; EXCEPTION WHEN duplicate_object THEN END \$\$" | sudo -u postgres psql  # no sensitive data here
echo "SELECT 'CREATE DATABASE gis OWNER gis' WHERE NOT EXISTS (SELECT FROM pg_database WHERE datname = 'gis')\gexec" | sudo -u postgres psql
echo "CREATE EXTENSION IF NOT EXISTS postgis;" | psql -h localhost gis gis
echo "CREATE EXTENSION IF NOT EXISTS hstore;" | psql -h localhost gis gis

## Removing obsolete python allows more of OpenStreetMap to load on the free instances
# rm -rf /usr/local/lib/python2.7


## Install python libraries

In [ ]:
try:
  import pydeck
  import psycopg2
except: 
  %pip install psycopg2-binary  
  %pip install pydeck

import pydeck
import html
import psycopg2
import pandas.io.sql 

## Download some OpenStreetMap data, and load it into postgres

In [ ]:
%%shell
wget -nc https://download.bbbike.org/osm/bbbike/PaloAlto/PaloAlto.osm.pbf
osm2pgsql PaloAlto.osm.pbf --host=localhost --user=gis --hstore --slim --database=gis

--2021-10-18 23:12:50--  https://download.bbbike.org/osm/bbbike/PaloAlto/PaloAlto.osm.pbf
Resolving download.bbbike.org (download.bbbike.org)... 116.202.174.6
Connecting to download.bbbike.org (download.bbbike.org)|116.202.174.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17900628 (17M) [application/x-pbf]
Saving to: ‘PaloAlto.osm.pbf’

PaloAlto.osm.pbf    100%[===================>]  17.07M  13.0MB/s    in 1.3s    

2021-10-18 23:12:52 (13.0 MB/s) - ‘PaloAlto.osm.pbf’ saved [17900628/17900628]



## Query for points nearby other points

In [166]:
with psycopg2.connect("dbname=gis user=gis host=localhost password=gis") as conn:
  sql = """
      select name,
             st_x(st_transform(way,4326)) as lng,
             st_y(st_transform(way,4326)) as lat,
             coalesce(name,
                tags -> 'species',
                amenity,
                "natural",
                tourism,
                barrier,
                highway,
                leisure,
                railway,
                man_made,
                tags -> 'seamark:type',
                tags -> 'traffic_calming',
                tags -> 'maxspeed',
                tags -> 'emergency',
                power,
                tags::text
              )   as info
             -- ,tags,*
        from planet_osm_point 
        where 
        
        ST_Distance(
		    	ST_Transform('SRID=4326;POINT(-122.166 37.428)'::geometry, 2163),
			    ST_Transform(way, 2163)
		    ) < 3000
        -- where name is not null
        --limit 10000
  """
  pdf = pandas.io.sql.read_sql_query(sql, conn)
pdf.values[80]
pdf.T[80].to_dict()

{'name': None,
 'lng': -122.168910782994,
 'lat': 37.4135712955336,
 'info': 'shrub'}

## Display the results

In [168]:
# Working blendfunc
#    https://deck.gl/docs/api-reference/json/conversion-reference

import pydeck
import pydeck as pdk

layer = pdk.Layer(
    "ScatterplotLayer",
    pdf,
    pickable=True,
    opacity=1,
    stroked=False,
    filled=True,
    radius_scale=10000,
    radius_min_pixels=1,
    radius_max_pixels=3,
    line_width_min_pixels=1,
    get_position=['lng', 'lat'],
    #get_position="coordinates",
    get_radius="exits_radius",
    get_fill_color=[196, 16, 4],
    get_line_color=[0, 0, 0],
    parameters= {
        'blend': True,
        'depthTest': False,
        'blendFunc': [770, 772]   # pydeck makes it really hard to find these enums.  
      }
)

# Set the viewport location
view_state = pdk.ViewState(
    longitude=-122.166,
    latitude=37.428,
    zoom=12,
    min_zoom=0,
    max_zoom=18,
    pitch=0,
    bearing=0)

r = pdk.Deck(layers=[layer], initial_view_state=view_state)
h = r.to_html(as_string=True,notebook_display=False)

r.show()
h2 = f"""<iframe srcdoc="{html.escape(h)}" style="height:600px; width:90%">"""
IPython.display.HTML(h2)
